In [73]:

import pandas as pd
from utils import get_engine
from dotenv import load_dotenv
from utils import get_df_from_db, interpret_model

In [74]:
load_dotenv()
engine = get_engine(echo_arg=True)

In [75]:
df = get_df_from_db(engine)
df.columns

2023-06-23 12:39:14,445 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-06-23 12:39:14,448 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-23 12:39:14,494 INFO sqlalchemy.engine.Engine select current_schema()
2023-06-23 12:39:14,497 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-23 12:39:14,541 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-06-23 12:39:14,543 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-23 12:39:14,587 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 12:39:14,597 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

Index(['YearBuilt', 'BuildingType', 'Neighborhood', 'Have_Stream_Energy',
       'Have_Electricity_Energy', 'Have_NaturalGas_Energy',
       'PrimaryPropertyType', 'NumberofBuildings', 'LargestPropertyUseTypeGFA',
       'TotalGHGEmissions', 'SiteEnergyUse_kBtu_',
       'LargestPropertyUseTypeGFA_log', 'TotalGHGEmissions_log',
       'SiteEnergyUse_kBtu_log'],
      dtype='object')

In [76]:
from sklearn.model_selection import train_test_split

In [77]:
y1, y2 = df['TotalGHGEmissions_log'], df['SiteEnergyUse_kBtu_log']
X_cols = ['Have_Stream_Energy', 'Have_NaturalGas_Energy', 'PrimaryPropertyType', 
          'LargestPropertyUseTypeGFA_log']
X = df[X_cols]

In [78]:
X.shape

(3314, 4)

In [79]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from interpret.glassbox import ExplainableBoostingRegressor, RegressionTree, LinearRegression

ebm = ExplainableBoostingRegressor()
ebm2 = RegressionTree()
ebm3 = LinearRegression()
X_cat = X.select_dtypes(include=[object, bool])
X_num = X.select_dtypes(exclude=[object, bool])
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.2, random_state=42)
preparation = ColumnTransformer(transformers=[
    ('tf_cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), X_cat.columns),
    ('tf_num', RobustScaler(), X_num.columns)
])

In [80]:
from interpret import show
X_tf_train = preparation.fit_transform(X_train)
X_tf_test = preparation.fit_transform(X_test)
names = preparation.get_feature_names_out(X_cols)
print(X_tf_test.shape)

df_train = pd.DataFrame(data=X_tf_train, columns=names)
df_test = pd.DataFrame(data=X_tf_test, columns=names)

models = [ebm, ebm2, ebm3]

for model in models:
    interpret_model(model, names, df_train, y_train, df_test, y_test)

(663, 29)


<!-- http://127.0.0.1:7633/140183493889920/ -->
 Open in new window

<!-- http://127.0.0.1:7633/140183490490944/ -->
 Open in new window

<!-- http://127.0.0.1:7633/140183489553536/ -->
 Open in new window